In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
import scienceplots

import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
plt.style.use('science')

# Import main utility functions
sys.path.insert(0, rf'{REPO_PATH}src')
from utils.main_utils import load_processed, load_variables

In [ ]:
FUTURES = ['CLc1', 'LCOc1']

VIEW = 2000

dfs = load_processed(FUTURES)

df = dfs['CLc1']

var_config = load_variables()
index_sets = {
    'S1': var_config['S1'],
    'S2': var_config['S2'],
    'S3': var_config['S3'],
    'S4': var_config['S4']
}

colors = sns.color_palette('bright', n_colors=5)

fig, axs = plt.subplots(2, 2, figsize=(10, 7), dpi=200, sharex=True)
axs = axs.flatten()

for i, (key, fetaures) in enumerate(index_sets.items()):
    ix_df = df[fetaures][-5500:VIEW-5500]
    ix_df.plot(ax=axs[i], color=colors, lw=0.8)

    axs[i].legend(loc='upper left', ncols=2, fontsize=10)
    axs[i].set_ylim(top=1.5 * axs[i].get_ylim()[1])
    axs[i].set_ylabel('Sentiment Intensity', fontsize=12)

fig.tight_layout(h_pad=-0.16)

fig.savefig(f'images/sentiment_indices.png')